In [4]:
!pip install dash pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 50.8 MB/s eta 0:00:00


In [6]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2025-10-08 09:57:12--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-10-08 09:57:12 (311 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [8]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the data
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# App layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Dropdown
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder="Select a Launch Site",
        searchable=True
    ),
    html.Br(),

    # Pie chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (kg):"),

    # Range slider
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        value=[min_payload, max_payload],
    ),

    # Scatter chart
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# ---- TASK 2: Pie chart callback ----
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df

    # ALL sites → total successes by site
    if entered_site == 'ALL':
        successes_by_site = (
            filtered_df[filtered_df['class'] == 1]
            .groupby('Launch Site')
            .size()
            .reset_index(name='Success Count')
        )
        fig = px.pie(
            successes_by_site,
            values='Success Count',
            names='Launch Site',
            title='Total Successful Launches by Site'
        )
        return fig
    else:
        # Specific site → success vs failure
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        outcome_counts = (
            site_df['class']
            .value_counts()
            .rename_axis('Outcome')
            .reset_index(name='Count')
        )
        outcome_counts['Outcome'] = outcome_counts['Outcome'].map({1: 'Success', 0: 'Failure'})
        fig = px.pie(
            outcome_counts,
            values='Count',
            names='Outcome',
            title=f'Launch Outcomes for {entered_site}'
        )
        return fig

# ---- TASK 4: Scatter chart callback ----
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_payload_scatter(selected_site, payload_range):
    # payload_range: [low, high]
    if payload_range and len(payload_range) == 2:
        low, high = payload_range
    else:
        low, high = min_payload, max_payload

    # Filter by payload range
    df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) &
        (spacex_df['Payload Mass (kg)'] <= high)
    ]

    # Filter by site if not ALL
    if selected_site and selected_site != 'ALL':
        df = df[df['Launch Site'] == selected_site]
        title = f'Payload vs Outcome for {selected_site} ({int(low)}–{int(high)} kg)'
    else:
        title = f'Payload vs Outcome for All Sites ({int(low)}–{int(high)} kg)'

    fig = px.scatter(
        df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        hover_data=['Launch Site', 'Payload Mass (kg)', 'class'],
        title=title
    )
    fig.update_yaxes(tickmode='array', tickvals=[0, 1], ticktext=['Failure', 'Success'])
    return fig

# Run the app
if __name__ == '__main__':
    app.run()


<IPython.core.display.Javascript object>